In [90]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

# Neo4j Import of GraphRAG Result Parquet files

This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization. 

You can also build your own GenAI applications using Neo4j and a number of RAG strategies with LangChain, LlamaIndex, Haystack, and many other frameworks.
See: https://neo4j.com/labs/genai-ecosystem

Here is what the end result looks like:

![](https://dev.assets.neo4j.com/wp-content/uploads/graphrag-neo4j-visualization.png)

## How does it work?

The notebook loads the parquet files from the `output` folder of your indexing process and loads them into Pandas dataframes.
It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships. 

All operations use MERGE, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement

```cypher
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [91]:
GRAPHRAG_FOLDER = "output/56789"

### Depedendencies

We only need Pandas and the neo4j Python driver with the rust extension for faster network transport.

In [92]:
%pip install --quiet pandas neo4j-rust-ext

Note: you may need to restart the kernel to use updated packages.


In [93]:
import time

import pandas as pd
from neo4j import GraphDatabase

## Neo4j Installation

You can create a free instance of Neo4j [online](https://console.neo4j.io). You get a credentials file that you can use for the connection credentials. You can also get an instance in any of the cloud marketplaces.

If you want to install Neo4j locally either use [Neo4j Desktop](https://neo4j.com/download) or 
the official Docker image: `docker run -e NEO4J_AUTH=neo4j/password -p 7687:7687 -p 7474:7474 neo4j` 

In [94]:
NEO4J_URI = "neo4j://localhost"  # or neo4j+s://xxxx.databases.neo4j.io
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "Jayant#4412"  # your password
NEO4J_DATABASE = "neo4j"

# Create a Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

## Batched Import

The batched import function takes a Cypher insert statement (needs to use the variable `value` for the row) and a dataframe to import.
It will send by default 1k rows at a time as query parameter to the database to be inserted.

In [95]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.

    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0, total, batch_size):
        batch = df.iloc[start : min(start + batch_size, total)]
        result = driver.execute_query(
            "UNWIND $rows AS value " + statement,
            rows=batch.to_dict("records"),
            database_=NEO4J_DATABASE,
        )
        print(result.summary.counters)
    print(f"{total} rows in {time.time() - start_s} s.")
    return total

## Indexes and Constraints

Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect.
Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`
* `__Covariate__`

In [97]:
# create constraints, idempotent operation

statements = [
    "\ncreate constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique",
    "\ncreate constraint document_id if not exists for (d:__Document__) require d.id is unique",
    "\ncreate constraint entity_id if not exists for (c:__Community__) require c.community is unique",
    "\ncreate constraint entity_id if not exists for (e:__Entity__) require e.id is unique",
    "\ncreate constraint entity_title if not exists for (e:__Entity__) require e.name is unique",
    "\ncreate constraint entity_title if not exists for (e:__Covariate__) require e.title is unique",
    "\ncreate constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique",
    "\n",
]

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


## Import Process

### Importing the Documents

We're loading the parquet file for the documents and create nodes with their ids and add the title property.
We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

In [98]:
pip install fastparquet pyarrow

Note: you may need to restart the kernel to use updated packages.


In [100]:
from fastparquet import ParquetFile
import pandas as pd

pf = ParquetFile(f"{GRAPHRAG_FOLDER}/documents.parquet")
doc_df = pf.to_pandas(columns=["id", "title"])


In [101]:
# Import documents
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title}
"""

batched_import(statement, doc_df)

SummaryCounters{labels_added: 7, nodes_created: 7, properties_set: 14, contains_updates: True, contains_system_updates: False}
7 rows in 0.09384393692016602 s.


7

### Loading Text Units

We load the text units, create a node per id and set the text and number of tokens.
Then we connect them to the documents that we created before.

In [102]:
text_df_temp = ParquetFile(
    f"{GRAPHRAG_FOLDER}/text_units.parquet")
text_df = text_df_temp.to_pandas(columns=["id", "text", "n_tokens", "document_ids"])
text_df.head(2)

,id,text,n_tokens,document_ids
0,9463933218712430a181b780bdadcd1d1c0975f4a703eb...,OPIOID QUANT\n\nBatch ID:\n\nAnalyst Run Revie...,1200,[1c21056c425a5db75ed855dfa306c8c5201ca88143e8a...
1,808b9c1cc710d9a25fd9ed887f147378d4dff6a687eef7...,) | NOS? AVLN | yi\nPol'eZO_%2 _O2077L _L do_ ...,1200,[1c21056c425a5db75ed855dfa306c8c5201ca88143e8a...


In [103]:
statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

SummaryCounters{labels_added: 447, relationships_created: 447, nodes_created: 447, properties_set: 1341, contains_updates: True, contains_system_updates: False}
447 rows in 0.5420017242431641 s.


447

### Loading Nodes

For the nodes we store id, name, description, embedding (if available), human readable id.

In [104]:
entity_df = ParquetFile(f"{GRAPHRAG_FOLDER}/entities.parquet").to_pandas()

entity_df["name"] = str(entity_df["human_readable_id"])
entity_df["description_embedding"] = 0
entity_df.head(2)

,id,human_readable_id,title,type,description,text_unit_ids,frequency,degree,x,y,name,description_embedding
0,5a49e166-4d12-4fab-9301-be72857424be,0,ANUJA BHARADWAJ,PERSON,Anuja Bharadwaj is a Forensic Science Examiner...,[9463933218712430a181b780bdadcd1d1c0975f4a703e...,5,5,0.0,0.0,0 0\n1 1\n2 2\n3 3...,0
1,52be4a17-b04c-42a7-b172-22f3e97c71b8,1,OPIOID QUANT,EVENT,OPIOID QUANT refers to a batch or process rela...,[9463933218712430a181b780bdadcd1d1c0975f4a703e...,1,3,0.0,0.0,0 0\n1 1\n2 2\n3 3...,0


In [109]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET
    e.human_readable_id = toString(value.human_readable_id),
    e.description = coalesce(value.description, ""),
    e.name = replace(coalesce(toString(value.name), ""), '"', "")
WITH e, value
// ✅ Only set the vector if it is a valid list of numbers
CALL {
    WITH e, value
    WITH e, value.description_embedding AS vec
    WHERE vec IS NOT NULL AND size(vec) > 0
    CALL db.create.setNodeVectorProperty(e, "description_embedding", vec)
    RETURN count(*) AS _
}
WITH e, value
UNWIND coalesce(value.text_unit_ids, []) AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)


"""

batched_import(entity_statement, entity_df)

ConstraintError: {neo4j_code: Neo.ClientError.Schema.ConstraintValidationFailed} {message: Node(457) already exists with label `__Entity__` and property `name` = '0        0
1        1
2        2
3        3
4        4
      ... 
926    926
927    927
928    928
929    929
930    930
Name: human_readable_id, Length: 931, dtype: int64'} {gql_status: 22N79} {gql_status_description: error: data exception - property uniqueness constraint violated. Property uniqueness constraint violated: Node(457) already exists with label `__Entity__` and property `name` = '0        0
1        1
2        2
3        3
4        4
      ... 
926    926
927    927
928    928
929    929
930    930
Name: human_readable_id, Length: 931, dtype: int64'.}

### Import Relationships

For the relationships we find the source and target node by name, using the base `__Entity__` type.
After creating the `RELATED` relationships, we set the description as attribute.

In [113]:
rel_df = ParquetFile(
    f"{GRAPHRAG_FOLDER}/relationships.parquet"
).to_pandas()
rel_df.head(2)

,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
0,654cd82a-709a-4474-a3a4-e2dcaac4727d,0,ANUJA BHARADWAJ,OPIOID QUANT,Anuja Bharadwaj is involved in the analysis an...,1.0,8,[9463933218712430a181b780bdadcd1d1c0975f4a703e...
1,520635cd-f5af-4115-a675-448e10481330,1,OPIOID QUANT,TEXAS,The OPIOID QUANT batch analysis might be takin...,5.0,4,[9463933218712430a181b780bdadcd1d1c0975f4a703e...


In [114]:
rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

SummaryCounters{contains_updates: False, contains_system_updates: False}
SummaryCounters{contains_updates: False, contains_system_updates: False}
1613 rows in 2.732862949371338 s.


1613

### Importing Communities

For communities we import their id, title, level.
We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [116]:
community_df = ParquetFile(
    f"{GRAPHRAG_FOLDER}/communities.parquet",
).to_pandas()

community_df.head(2)

,id,human_readable_id,community,level,parent,children,title,entity_ids,relationship_ids,text_unit_ids,period,size
0,221c0159-139f-493f-9ec9-d5dab4fa82d9,0,0,0,-1,"[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2...",Community 0,"[da00b277-f6f4-4b7c-b1ee-e75800a44349, a652d52...","[0aeb00c4-e03d-4967-85d9-2bf100da9402, 0c4d2a0...",[050f3e50c1ed03365c2c17cf3e58bb04ab60ebc624875...,2025-10-14,93
1,6b75a8e4-370a-4717-8315-56b9ca7831be,1,1,0,-1,"[32, 33, 34, 35, 36, 37, 38]",Community 1,"[d616508b-b13d-418d-adc0-f0013fe5cd80, 88071d3...","[02fdbe55-0704-413c-b9f2-9e61103001b8, 09929ab...",[01fbb8fc4a2ceaa7e244d32402bb5604ba276d5dfe84c...,2025-10-14,59


In [117]:
statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURn count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

SummaryCounters{labels_added: 201, nodes_created: 201, properties_set: 603, contains_updates: True, contains_system_updates: False}
201 rows in 0.5933513641357422 s.


201

### Importing Community Reports

Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about.
For the findings we create the findings in context of the communities.

In [123]:
community_report_df = ParquetFile(
    f"{GRAPHRAG_FOLDER}/community_reports.parquet"
).to_pandas()
community_report_df["findings"] = community_report_df["findings"].apply(
    lambda x: x if isinstance(x, list) and len(x) > 0 else []
)

community_report_df.head(2)

,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
0,167889cd5508414d9125914b09810a3e,199,199,4,196,[],Henry Sarbieski and the Route 42 Incident,"The community centers around Henry Sarbieski, ...",# Henry Sarbieski and the Route 42 Incident\n\...,6.5,The impact severity rating is moderate due to ...,[],"{\n ""title"": ""Henry Sarbieski and the Route...",2025-10-14,11
1,5fc39e9f8c1c4babab6538f4c8a8ece6,200,200,4,196,[],Investigation of Henry Sarbieski by Ryan P Gog...,The community centers around the investigation...,# Investigation of Henry Sarbieski by Ryan P G...,4.5,The impact severity rating is moderate due to ...,[],"{\n ""title"": ""Investigation of Henry Sarbie...",2025-10-14,2


In [124]:
# Import communities
community_statement = """
MERGE (c:__Community__ {community:value.community})
SET c += value {.level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_statement, community_report_df)

SummaryCounters{labels_added: 201, nodes_created: 201, properties_set: 1206, contains_updates: True, contains_system_updates: False}
201 rows in 0.181473970413208 s.


201

### Importing Covariates

Covariates are for instance claims on entities, we connect them to the chunks where they originate from.

In [125]:
cov_df = ParquetFile(f"{GRAPHRAG_FOLDER}/covariates.parquet")
#                         columns=["id","text_unit_id"])
cov_df.head(2)
# Subject id do not match entity ids

FileNotFoundError: output/56789/covariates.parquet

In [ ]:
# Import covariates
cov_statement = """
MERGE (c:__Covariate__ {id:value.id})
SET c += apoc.map.clean(value, ["text_unit_id", "document_ids", "n_tokens"], [NULL, ""])
WITH c, value
MATCH (ch:__Chunk__ {id: value.text_unit_id})
MERGE (ch)-[:HAS_COVARIATE]->(c)
"""
batched_import(cov_statement, cov_df)

{'_contains_updates': True, 'labels_added': 89, 'relationships_created': 89, 'nodes_created': 89, 'properties_set': 1061}
89 rows in 0.13370895385742188 s.


89

### Visualize your data

You can now [Open] Neo4j on Aura, you need to log in with either SSO or your credentials.

Or open https://workspace-preview.neo4j.io and connect to your local instance, remember the URI is `neo4j://localhost` and `neo4j` as username and `password` as password.

In "Explore" you can explore by using visual graph patterns and then explore and expand further.

In "Query", you can open the left sidebar and explore by clicking on the nodes and relationships.
You can also use the co-pilot to generate Cypher queries for your, here are some examples.

#### Show a few `__Entity__` nodes and their relationships (Entity Graph)

```cypher
MATCH path = (:__Entity__)-[:RELATED]->(:__Entity__)
RETURN path LIMIT 200
```

#### Show the Chunks and the Document (Lexical Graph)

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(c:__Chunk__)
RETURN path LIMIT 100
```

####  Show a Community and it's Entities

```cypher
MATCH (c:__Community__) WITH c LIMIT 1
MATCH path = (c)<-[:IN_COMMUNITY]-()-[:RELATED]-(:__Entity__)
RETURN path LIMIT 100
```

#### Show everything

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(:__Chunk__)-[:HAS_ENTIY]->()-[:RELATED]-()-[:IN_COMMUNITY]->()
RETURN path LIMIT 250
```

We showed the visualization of this last query at the beginning.

If you have questions, feel free to reach out in the GraphRAG discord server: 
https://discord.gg/graphrag

In [137]:
community_df = ParquetFile(
    f"{GRAPHRAG_FOLDER}/communities.parquet"
).to_pandas()

community_df = community_df[community_df["human_readable_id"] == 195]
listl1 = community_df["entity_ids"]

In [141]:
print(list(listl1))

[['b220936c-edc2-4763-af3a-295121780a7f', '1ff60c3b-68f1-43e9-8f4a-275332636db5']]


In [144]:
entity_df = ParquetFile(
    f"{GRAPHRAG_FOLDER}/entities.parquet"
).to_pandas()
entity_df = entity_df[entity_df["id"]=="1ff60c3b-68f1-43e9-8f4a-275332636db5"]
entity_df

,id,human_readable_id,title,type,description,text_unit_ids,frequency,degree,x,y
727,1ff60c3b-68f1-43e9-8f4a-275332636db5,727,WINDSOR,GEO,Windsor is a town in Connecticut where the law...,[3bef833d46608ebfe01b71e198d898ea1120d8cdd1727...,1,1,0.0,0.0
